In [7]:
import pandas as pd
import matplotlib.pyplot as plt
df_bcn = pd.read_csv("../data/bcnJune_listings.csv")
df_mad= pd.read_csv("madJune_listings.csv")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# ------------------------------------------------------------
# 1. LIMPIEZA DE DATOS
# ------------------------------------------------------------

def clean_df(df):
    df = df.copy()

    # Nos quedamos solo con columnas necesarias
    cols = ["host_id", "id", "neighbourhood", "room_type"]
    df = df[cols]

    # Eliminamos filas con host_id o listing_id vacíos
    df = df.dropna(subset=["host_id", "id"])

    # Convertimos host_id a string (a veces viene como número muy grande y da problemas)
    df["host_id"] = df["host_id"].astype(str)

    return df


B_clean = clean_df(B)
OC_clean = clean_df(OC)

In [ ]:
 ------------------------------------------------------------
# 2. CALCULAR CUÁNTOS LISTINGS TIENE CADA HOST
# ------------------------------------------------------------

def compute_concentration(df):
    # Cuenta cuántos pisos tiene cada host
    host_counts = df.groupby("host_id")["id"].count().sort_values(ascending=False)

    # Calculamos porcentajes
    total_listings = host_counts.sum()
    host_pct = (host_counts / total_listings) * 100

    return host_counts, host_pct


B_counts, B_pct = compute_concentration(B_clean)
OC_counts, OC_pct = compute_concentration(OC_clean)

In [ ]:
# ------------------------------------------------------------
# 3. PREPARAR DATOS PARA EL GRÁFICO
# (TOP 20 hosts de cada ciudad)
# ------------------------------------------------------------

top_n = 20
B_top = B_counts.head(top_n)
OC_top = OC_counts.head(top_n)

In [ ]:
# ------------------------------------------------------------
# 4. GRAFICAR
# ------------------------------------------------------------

plt.figure(figsize=(14, 7))

plt.plot(
    range(1, top_n + 1),
    B_top.values,
    marker="o",
    label="Barcelona – Top Hosts"
)

plt.plot(
    range(1, top_n + 1),
    OC_top.values,
    marker="o",
    label="Otra Ciudad – Top Hosts"
)

plt.xlabel("Ranking del Host (1 = mayor número de listings)")
plt.ylabel("Número de propiedades")
plt.title("Concentración de propiedades entre los hosts más grandes")
plt.legend()
plt.grid(True)
plt.tight_layout()

plt.savefig("reports/graph3.png")
plt.close()

In [ ]:
# ------------------------------------------------------------
# 5. CONCLUSIÓN GENERADA AUTOMÁTICAMENTE
# ------------------------------------------------------------

def generate_conclusion(city_name, counts, pct):
    top1 = counts.iloc[0]
    top5 = counts.iloc[:5].sum()
    top10pct = pct.iloc[:10].sum()

    return (
        f"En {city_name}, el host más grande posee {top1} alojamientos. "
        f"Los 5 hosts principales acumulan {top5} alojamientos. "
        f"El top 10 concentra el {top10pct:.2f}% de toda la oferta, lo que indica "
        f"{'alta' if top10pct > 40 else 'baja'} concentración del mercado."
    )


print("\n--- CONCLUSIÓN ---\n")
print("Barcelona:")
print(generate_conclusion("Barcelona", B_counts, B_pct))
print("\nOtra Ciudad:")
print(generate_conclusion("la otra ciudad", OC_counts, OC_pct))
print("\nEl gráfico 'graph3.png' ha sido guardado en la carpeta reports/")